In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
tf.keras.backend.clear_session()

In [7]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [8]:
subject = 'Maranhão - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [9]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,anhã-03o - Produção de Cimento (t),Maranhão - IDH Renda,...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH,Maranhão - Consumo de Cimento (t)
0,2008,0.645315,7.978325,0.736322,4.598085e+07,2.691136e+06,6.129297,3.956333e+07,352.164879,0.592503,...,11.760983,25.354093,1004.987958,5445.077556,26.913732,23.175592,80.950690,1.958593,0.656244,963.205000
1,2009,0.650266,8.007891,0.737873,4.630053e+07,2.690232e+06,6.126304,3.956853e+07,363.707506,0.594639,...,9.472585,28.506238,1004.900392,3620.312713,26.992836,23.414460,81.602937,1.847599,0.658950,1123.586000
2,2010,0.654819,7.971707,0.739020,4.751040e+07,2.700684e+06,6.153755,3.988864e+07,424.052860,0.596141,...,9.332485,31.838993,1004.891838,2594.109885,26.968458,23.782363,83.292728,1.860782,0.661204,1378.741000
3,2011,0.658970,7.961524,0.739772,4.866181e+07,2.701549e+06,6.182820,4.018069e+07,558.996171,0.597021,...,10.985689,33.196582,1005.053412,2035.008505,27.022388,24.015667,84.183211,1.989042,0.663017,1282.016000
4,2012,0.662311,10.302202,0.740034,4.972690e+07,2.696497e+06,6.208137,4.043865e+07,688.169638,0.597229,...,8.159014,31.592321,1005.214749,1723.384057,26.989407,23.760558,83.318460,1.993051,0.664173,1528.770000
5,2013,0.666586,11.189661,0.740120,5.062157e+07,2.675613e+06,6.222354,4.060687e+07,719.208496,0.596832,...,7.913561,29.227149,1005.449976,1548.437409,26.912712,23.364264,81.867671,1.963286,0.665521,1563.125000
6,2014,0.668885,11.986521,0.739796,5.132137e+07,2.651392e+06,6.227127,4.069030e+07,687.203882,0.595984,...,10.350442,28.491261,1005.705374,1452.214268,27.001638,23.279428,81.037312,1.988340,0.665898,1303.411571
7,2015,0.671376,12.529442,0.739029,5.174856e+07,2.602821e+06,6.218491,4.066452e+07,630.565434,0.594816,...,12.454398,29.089818,1005.751406,1363.412394,27.085524,23.327394,80.918789,1.953057,0.666139,1191.509657
8,2016,0.688469,12.762358,0.746544,5.210281e+07,2.570101e+06,6.220890,4.069623e+07,538.170446,0.606351,...,13.111908,28.880429,1005.704586,1297.690355,27.029688,23.289212,81.028777,1.867218,0.677976,1100.217771
9,2017,0.689875,12.460135,0.745411,5.221165e+07,2.559679e+06,6.221665,4.069725e+07,440.490154,0.605565,...,9.479812,27.965594,1005.796545,1279.407347,26.893536,23.259717,81.515345,1.794104,0.677792,971.921000


In [10]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,anhã-03o - Produção de Cimento (t),Maranhão - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,0.645315,7.978325,0.736322,4.598085e+07,2.691136e+06,6.129297,3.956333e+07,352.164879,0.592503,5.483335e+08,...,2.868108,11.760983,25.354093,1004.987958,5445.077556,26.913732,23.175592,80.950690,1.958593,0.656244
1,0.650266,8.007891,0.737873,4.630053e+07,2.690232e+06,6.126304,3.956853e+07,363.707506,0.594639,5.302764e+08,...,2.305885,9.472585,28.506238,1004.900392,3620.312713,26.992836,23.414460,81.602937,1.847599,0.658950
2,0.654819,7.971707,0.739020,4.751040e+07,2.700684e+06,6.153755,3.988864e+07,424.052860,0.596141,5.046425e+08,...,1.958246,9.332485,31.838993,1004.891838,2594.109885,26.968458,23.782363,83.292728,1.860782,0.661204
3,0.658970,7.961524,0.739772,4.866181e+07,2.701549e+06,6.182820,4.018069e+07,558.996171,0.597021,4.854622e+08,...,2.079088,10.985689,33.196582,1005.053412,2035.008505,27.022388,24.015667,84.183211,1.989042,0.663017
4,0.662311,10.302202,0.740034,4.972690e+07,2.696497e+06,6.208137,4.043865e+07,688.169638,0.597229,4.867661e+08,...,2.183282,8.159014,31.592321,1005.214749,1723.384057,26.989407,23.760558,83.318460,1.993051,0.664173
5,0.666586,11.189661,0.740120,5.062157e+07,2.675613e+06,6.222354,4.060687e+07,719.208496,0.596832,5.225848e+08,...,2.403445,7.913561,29.227149,1005.449976,1548.437409,26.912712,23.364264,81.867671,1.963286,0.665521
6,0.668885,11.986521,0.739796,5.132137e+07,2.651392e+06,6.227127,4.069030e+07,687.203882,0.595984,5.676413e+08,...,2.748397,10.350442,28.491261,1005.705374,1452.214268,27.001638,23.279428,81.037312,1.988340,0.665898
7,0.671376,12.529442,0.739029,5.174856e+07,2.602821e+06,6.218491,4.066452e+07,630.565434,0.594816,5.179392e+08,...,2.501529,12.454398,29.089818,1005.751406,1363.412394,27.085524,23.327394,80.918789,1.953057,0.666139
8,0.688469,12.762358,0.746544,5.210281e+07,2.570101e+06,6.220890,4.069623e+07,538.170446,0.606351,4.524144e+08,...,2.073464,13.111908,28.880429,1005.704586,1297.690355,27.029688,23.289212,81.028777,1.867218,0.677976
9,0.689875,12.460135,0.745411,5.221165e+07,2.559679e+06,6.221665,4.069725e+07,440.490154,0.605565,4.213689e+08,...,2.177386,9.479812,27.965594,1005.796545,1279.407347,26.893536,23.259717,81.515345,1.794104,0.677792


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1123.586000
1     1378.741000
2     1282.016000
3     1528.770000
4     1563.125000
5     1303.411571
6     1191.509657
7     1100.217771
8      971.921000
9      750.012000
10     774.045000
11    1038.249000
12    1059.129000
13    1083.999000
14            NaN
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,anhã-03o - Produção de Cimento (t),Maranhão - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,-1.548004,-1.341931,-1.526781,-1.800669,0.888019,-1.813414,-1.863562,-1.169993,-1.353287,1.204752,...,2.026352,0.984445,-1.704426,-1.179358,2.780896,-0.581090,-1.106443,-1.032012,0.713734,-1.515780
1,-1.235625,-1.327021,-0.992611,-1.657451,0.872218,-1.896902,-1.851185,-1.085351,-0.893854,0.820657,...,0.085558,-0.051710,-0.236979,-1.419199,1.260440,0.474888,-0.140579,-0.401753,-0.347981,-1.152490
2,-0.948330,-1.345268,-0.597544,-1.115428,1.054815,-1.131157,-1.089628,-0.642834,-0.570965,0.275392,...,-1.114490,-0.115145,1.314548,-1.442629,0.405374,0.149463,1.347032,1.231074,-0.221880,-0.849804
3,-0.686418,-1.350403,-0.338594,-0.599596,1.069928,-0.320349,-0.394850,0.346715,-0.381737,-0.132596,...,-0.697344,0.633403,1.946559,-1.000083,-0.060488,0.869373,2.290396,2.091538,1.004993,-0.606270
4,-0.475631,-0.170051,-0.248616,-0.122433,0.981661,0.385888,0.218857,1.293953,-0.336916,-0.104860,...,-0.337667,-0.646476,1.199713,-0.558185,-0.320144,0.429102,1.258865,1.255939,1.043345,-0.450985
5,-0.205832,0.277475,-0.218954,0.278382,0.616846,0.782503,0.619058,1.521563,-0.422410,0.657046,...,0.422337,-0.757614,0.098633,0.086092,-0.465916,-0.594704,-0.343548,-0.145943,0.758629,-0.269993
6,-0.060778,0.679314,-0.330312,0.591894,0.193718,0.915640,0.817536,1.286871,-0.604630,1.615453,...,1.613108,0.345772,-0.243951,0.785621,-0.546092,0.592379,-0.686579,-0.948310,0.998286,-0.219397
7,0.096364,0.953097,-0.594440,0.783275,-0.654791,0.674736,0.756208,0.871538,-0.855798,0.558230,...,0.760923,1.298413,0.034700,0.911699,-0.620085,1.712186,-0.492629,-1.062838,0.660786,-0.186967
8,1.174906,1.070551,1.993279,0.941980,-1.226398,0.741645,0.831633,0.193999,1.624690,-0.835563,...,-0.716758,1.596124,-0.062778,0.783463,-0.674847,0.966821,-0.647020,-0.956557,-0.160319,1.402510
9,1.263639,0.918147,1.603303,0.990742,-1.408452,0.763263,0.834059,-0.522298,1.455606,-1.495939,...,-0.358019,-0.048438,-0.488670,1.035335,-0.690081,-0.850683,-0.766284,-0.486392,-0.859692,1.377796


In [13]:
reshaped_train_input = dfToInputRNN(train_input)

In [14]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 1.21119213,  0.75064964,  0.29363176,  0.73311646,
         -1.01788684,  0.18340237,  0.37196342, -1.03040072,
          1.03003753, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [15]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     1123.586000
1     1378.741000
2     1282.016000
3     1528.770000
4     1563.125000
5     1303.411571
6     1191.509657
7     1100.217771
8      971.921000
9      750.012000
10     774.045000
11    1038.249000
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [16]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    1038.249
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [17]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,anhã-03o - Produção de Cimento (t),Maranhão - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
12,1.137321,0.665597,-0.713268,0.506899,-0.617267,0.098822,0.211599,-1.026592,0.829595,1.108405,...,-1.482779,-2.260447,-0.619583,0.376717,-0.483958,-1.82319,-0.372439,0.371831,-1.374154,0.896376


In [18]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 1.13732127,  0.66559737, -0.71326822,  0.50689924,
         -0.61726733,  0.09882248,  0.21159948, -1.02659168,
          0.8295948 ,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [19]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1059.129
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [31]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [32]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [33]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1452796451, 2069943538, 790983133, 2618713314, 2932885658, 910005740, 2887020779, 644246711, 2907457302, 1010350772, 1346956573, 808364028, 1137401333, 2184414847, 1859003379, 66726756, 4215673007, 3646332395, 3453265405, 3782247219, 3380176993, 5199383, 3370312989, 244951903, 2597267697, 3810777144, 1736799655, 2363164684, 4226612346, 2863563312, 3297845178, 2464077595, 4294924922, 2026483915, 648150252, 3788026014, 14295888, 3762981296, 473239189, 2916302857, 476950661, 2340974832, 1516495950, 3615187198, 3781641658, 4242742229, 4090302006, 3117194056, 1805938158, 2414610912, 1680306830, 4133412013, 1585455579, 2576460160, 89826282, 1089406890, 361550721, 3132453147, 4004695504, 3547151727, 3087127258, 343741487, 259317395, 3985532646, 3188149580, 1092668023, 1563990467, 2920323854, 1813228522, 2748277750, 366949724, 2705080464, 1426186232, 4099998254, 2214018134, 610572974, 520710919, 494188516, 3820007088, 1441140670, 4170845384, 3733053746, 3161083535, 777674332, 2935409213, 1078

loss: 2348.47265625


Step: 89 ___________________________________________
loss: 1035.5513916015625


Step: 90 ___________________________________________
loss: 1732.3609619140625


Step: 91 ___________________________________________
loss: 1265.6490478515625


Step: 92 ___________________________________________
loss: 1046.6973876953125


Step: 93 ___________________________________________
loss: 1498.6197509765625


Step: 94 ___________________________________________
loss: 2017.5220947265625


Step: 95 ___________________________________________
loss: 3700.220458984375


Step: 96 ___________________________________________
loss: 1901.4932861328125


Step: 97 ___________________________________________
loss: 2683.451171875


Step: 98 ___________________________________________
loss: 1699.5169677734375


Step: 99 ___________________________________________
loss: 969.6467895507812


final_seed: 5199383


In [34]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 1425057.0000
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 1393409.5000
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 1225816.1250
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 303684.9688
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 98541.5000
Epoch 6/10000
1/1 [==============================] - 0s 476ms/step - loss: 60304.2812
Epoch 7/10000
1/1 [==============================] - 0s 7ms/step - loss: 43169.3711
Epoch 8/10000
1/1 [==============================] - 0s 6ms/step - loss: 27025.4668
Epoch 9/10000
1/1 [==============================] - 0s 6ms/step - loss: 21802.0879
Epoch 10/10000
1/1 [==============================] - 0s 8ms/step - loss: 19859.8867
Epoch 11/10000
1/1 [==============================] - 0s 7ms/step - loss: 19776.8711
Epoch 12/10000
1/1 [==============================] - 0s 6ms/step 

1/1 [==============================] - 0s 4ms/step - loss: 15261.9570
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 16689.7676
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 26815.5703
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 13194.2334
Epoch 101/10000
1/1 [==============================] - 0s 3ms/step - loss: 14102.1162
Epoch 102/10000
1/1 [==============================] - 0s 3ms/step - loss: 14345.0107
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 16444.5820
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 33278.8594
Epoch 105/10000
1/1 [==============================] - 0s 3ms/step - loss: 17481.3438
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 27036.4590
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 23740.3184
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 12360.8994
Epoch 193/10000
1/1 [==============================] - 0s 4ms/step - loss: 14721.8721
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 14687.3408
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 25973.9844
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 25976.5098
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 14856.7568
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 19476.0703
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 27810.1797
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 22364.7266
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 19786.0586
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 21549.3984
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 22123.7266
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 9917.2432
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 20910.9395
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 15962.2607
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 25051.8418
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 21200.1113
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 12913.1533
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 9714.9365
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 11608.1094
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 17378.3027
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 11642.6484
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 6875.1914
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 7682.1938
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 12072.3584
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 20421.9434
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 9613.9912
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 8996.5176
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 18425.1895
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 9448.9912
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 13206.3643
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 17170.7383
Epoch 392/10000
1/1 [==============================] - 0s 3ms/step - loss: 9156.0361
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - loss: 2

...

1/1 [==============================] - 0s 4ms/step - loss: 6399.6035
Epoch 1430/10000
1/1 [==============================] - 0s 4ms/step - loss: 14675.7451
Epoch 1431/10000
1/1 [==============================] - 0s 4ms/step - loss: 6502.8384
Epoch 1432/10000
1/1 [==============================] - 0s 4ms/step - loss: 13626.7100
Epoch 1433/10000
1/1 [==============================] - 0s 4ms/step - loss: 7740.8237
Epoch 1434/10000
1/1 [==============================] - 0s 4ms/step - loss: 6689.3853
Epoch 1435/10000
1/1 [==============================] - 0s 4ms/step - loss: 10766.4697
Epoch 1436/10000
1/1 [==============================] - 0s 4ms/step - loss: 5216.3906
Epoch 1437/10000
1/1 [==============================] - 0s 4ms/step - loss: 19793.2012
Epoch 1438/10000
1/1 [==============================] - 0s 5ms/step - loss: 7758.9375
Epoch 1439/10000
1/1 [==============================] - 0s 5ms/step - loss: 3876.0195
Epoch 1440/10000
1/1 [==============================] - 0s 4ms/step

1/1 [==============================] - 0s 3ms/step - loss: 8320.4971
Epoch 1525/10000
1/1 [==============================] - 0s 4ms/step - loss: 9015.7002
Epoch 1526/10000
1/1 [==============================] - 0s 4ms/step - loss: 8875.4688
Epoch 1527/10000
1/1 [==============================] - 0s 4ms/step - loss: 6789.2925
Epoch 1528/10000
1/1 [==============================] - 0s 4ms/step - loss: 8163.2222
Epoch 1529/10000
1/1 [==============================] - 0s 4ms/step - loss: 8781.8584
Epoch 1530/10000
1/1 [==============================] - 0s 4ms/step - loss: 15947.3408
Epoch 1531/10000
1/1 [==============================] - 0s 4ms/step - loss: 8109.7856
Epoch 1532/10000
1/1 [==============================] - 0s 5ms/step - loss: 5018.9038
Epoch 1533/10000
1/1 [==============================] - 0s 4ms/step - loss: 5791.9243
Epoch 1534/10000
1/1 [==============================] - 0s 4ms/step - loss: 9828.1494
Epoch 1535/10000
1/1 [==============================] - 0s 4ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 5271.6587
Epoch 1620/10000
1/1 [==============================] - 0s 4ms/step - loss: 8690.3965
Epoch 1621/10000
1/1 [==============================] - 0s 4ms/step - loss: 4951.5425
Epoch 1622/10000
1/1 [==============================] - 0s 4ms/step - loss: 2653.7263
Epoch 1623/10000
1/1 [==============================] - 0s 4ms/step - loss: 4651.8394
Epoch 1624/10000
1/1 [==============================] - 0s 4ms/step - loss: 4442.4863
Epoch 1625/10000
1/1 [==============================] - 0s 4ms/step - loss: 7801.4858
Epoch 1626/10000
1/1 [==============================] - 0s 4ms/step - loss: 6641.4565
Epoch 1627/10000
1/1 [==============================] - 0s 4ms/step - loss: 6409.9312
Epoch 1628/10000
1/1 [==============================] - 0s 4ms/step - loss: 9891.9814
Epoch 1629/10000
1/1 [==============================] - 0s 4ms/step - loss: 8950.2061
Epoch 1630/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 3784.4336
Epoch 1715/10000
1/1 [==============================] - 0s 4ms/step - loss: 8260.9893
Epoch 1716/10000
1/1 [==============================] - 0s 4ms/step - loss: 6935.5923
Epoch 1717/10000
1/1 [==============================] - 0s 4ms/step - loss: 5528.5391
Epoch 1718/10000
1/1 [==============================] - 0s 4ms/step - loss: 10481.9697
Epoch 1719/10000
1/1 [==============================] - 0s 3ms/step - loss: 4112.3560
Epoch 1720/10000
1/1 [==============================] - 0s 3ms/step - loss: 4624.9614
Epoch 1721/10000
1/1 [==============================] - 0s 3ms/step - loss: 7613.2681
Epoch 1722/10000
1/1 [==============================] - 0s 4ms/step - loss: 6928.2266
Epoch 1723/10000
1/1 [==============================] - 0s 4ms/step - loss: 4934.6826
Epoch 1724/10000
1/1 [==============================] - 0s 4ms/step - loss: 4481.0898
Epoch 1725/10000
1/1 [==============================] - 0s 4ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 8383.9893
Epoch 1810/10000
1/1 [==============================] - 0s 4ms/step - loss: 5941.4546
Epoch 1811/10000
1/1 [==============================] - 0s 4ms/step - loss: 8395.1680
Epoch 1812/10000
1/1 [==============================] - 0s 4ms/step - loss: 8838.2705
Epoch 1813/10000
1/1 [==============================] - 0s 4ms/step - loss: 3654.3477
Epoch 1814/10000
1/1 [==============================] - 0s 4ms/step - loss: 10093.1963
Epoch 1815/10000
1/1 [==============================] - 0s 4ms/step - loss: 6586.9810
Epoch 1816/10000
1/1 [==============================] - 0s 4ms/step - loss: 12235.7422
Epoch 1817/10000
1/1 [==============================] - 0s 4ms/step - loss: 8565.6396
Epoch 1818/10000
1/1 [==============================] - 0s 4ms/step - loss: 6120.1255
Epoch 1819/10000
1/1 [==============================] - 0s 4ms/step - loss: 10084.7002
Epoch 1820/10000
1/1 [==============================] - 0s 13ms/step

In [35]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 637ms/step
[1059.129](test_target) - [994.52954](prediction) = 64.5994589843749


In [36]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.060993003670350744

In [37]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [38]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,anhã-03o - Produção de Cimento (t),Maranhão - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,-1.241459,-0.486246,-1.281082,-0.935735,-0.609310,-0.581902,-0.724135,-0.882250,-1.289469,1.148205,...,1.308949,1.412346,-1.213218,1.409635,1.321922,-1.348681,-1.129447,-1.011331,1.405833,-1.260158
1,0.034142,1.393198,0.121772,-0.450446,-0.800585,-0.825312,-0.689940,-0.516072,0.141790,0.140893,...,-0.190804,-0.643250,-0.022737,-0.606347,-0.225769,1.042844,-0.172339,-0.350435,-0.836052,0.074200
2,1.207317,-0.906952,1.159310,1.386181,1.409895,1.407214,1.414076,1.398322,1.147679,-1.289098,...,-1.118144,-0.769096,1.235955,-0.803288,-1.096153,0.305837,1.301786,1.361766,-0.569781,1.185958


train_target:


,Maranhão - Consumo de Cimento (t)
0,1123.586
1,1378.741
2,1282.016


test_input:


...

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,anhã-03o - Produção de Cimento (t),Maranhão - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
12,1.137321,0.665597,-0.713268,0.506899,-0.617267,0.098822,0.211599,-1.026592,0.829595,1.108405,...,-1.482779,-2.260447,-0.619583,0.376717,-0.483958,-1.82319,-0.372439,0.371831,-1.374154,0.896376


test_target:


,Maranhão - Consumo de Cimento (t)
12,1059.129


1/1 [==============================] - 1s 577ms/step
Error: 64.5994589843749




[1356.4990234375,
 1448.665771484375,
 1364.6864013671875,
 1250.940673828125,
 1244.0909423828125,
 1112.01611328125,
 986.4115600585938,
 761.7857055664062,
 769.609375,
 994.529541015625]

In [39]:
display(targets)
display(predictions)

[1528.77,
 1563.125,
 1303.4115714285715,
 1191.5096571428571,
 1100.2177714285715,
 971.921,
 750.012,
 774.045,
 1038.249,
 1059.129]

[1356.4990234375,
 1448.665771484375,
 1364.6864013671875,
 1250.940673828125,
 1244.0909423828125,
 1112.01611328125,
 986.4115600585938,
 761.7857055664062,
 769.609375,
 994.529541015625]

In [40]:
mae = mean_absolute_error(predictions, targets)
mae

127.33022744140621

In [41]:
porcentage = mae/np.mean(targets)
porcentage

0.11287750462652994